## Weather API code form Open Meteo
https://open-meteo.com/en/docs/historical-weather-api/

## READ ME
To have the machine learning model make predictions using weather it needs to have been trained with the weather data as one of the inputs. The weather api below gets the weather history bewtween a range of dates and sends it to a dataframe.

In [ ]:
!pip install openmeteo-requests requests-cache retry-requests numpy pandas
!pip install requests
import requests
import pandas as pd
import numpy as np

# READ ME
This is not the final version of the training data, so you cannot rely on copying and pasting the first and last date from it or that the dates wont change

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


youll need to change the first part of this file path

In [ ]:
csv_files = ['/content/drive/Shareddrives/Untitled shared drive/50_ResidentialPowerDisaggregation_SD_Fall23/1.2 Software/Colab Notebooks/Weather API/Copy of MLData0.csv']

# Load and concatenate data from multiple CSV files if needed
data_list = []
for csv_file in csv_files:
    data = pd.read_csv(csv_file)
    data_list.append(data)

# Concatenate data from all CSV files
training_data = pd.concat(data_list, ignore_index=True)

In [ ]:
training_data

,timestamp,Total,Washer,BlowerGH,Lights,BlowerBed,CompGH,CompBed,Dryer,Recs1,Recs2,WaterHeater
0,1/18/2023 22:00,164.1,0.5,4.6,25.7,1.7,21.7,5.8,0.0,7.1,5.6,0.0
1,1/18/2023 22:01,164.3,0.5,4.6,25.6,1.7,21.7,5.8,0.0,7.1,5.6,0.0
2,1/18/2023 22:02,164.1,0.6,4.6,25.8,1.8,21.7,5.8,0.0,7.0,5.5,0.0
3,1/18/2023 22:03,164.0,0.5,4.5,25.6,1.8,21.6,5.8,0.0,7.1,5.6,0.0
4,1/18/2023 22:04,164.2,0.6,4.6,25.7,1.7,21.7,5.8,0.0,7.0,5.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5771,1/22/2023 22:11,7091.1,6.4,8.9,41.7,225.5,25.2,629.4,970.1,10.1,7.3,0.0
5772,1/22/2023 22:12,1931.8,6.0,6.3,43.4,226.8,21.1,628.1,-0.1,7.3,5.8,0.0
5773,1/22/2023 22:13,1943.2,6.0,6.4,43.5,227.9,21.2,631.0,-0.1,7.3,5.8,0.0
5774,1/22/2023 22:14,7393.9,6.4,9.2,41.9,225.9,25.6,628.0,1026.7,10.3,7.5,0.0


In [ ]:
s d

# READ ME
Above is an example of what the training data will look like. The only relevant column for adding the weather history is the timestamp. this was done by collecting weather history from the https://open-meteo.com/ database. One way of adding the weather history would be to do the following:


*   start date in the code below would become a varaible with the earliest timestamp from the training data
*   end date in the code below would become a variable with the lastest date from the training data
*   (This was done dynamically with code instead of typing out what the timestamp is. With more training data the first and last timestamp could be different)  
*   The weather API will create a dataframe with two columns, the timestamp, and the temperature, however it is only updated every hour so the dataframe sizes aren't the same
*   You can add the weather column to the training data, but since it is only from every hour, minutes 1-60 would have the same temperature value
*   You can compare the first part of the timestamp in the training data to the first part of the timestamp for the weather data, and if they are the same you add that temperature value to the training data


Example:
Training data has timestamps 1/18/23 22:00, 1/18/23 22:01, 1/18/23 22:02, 1/18/23 22:03,... up to 1/18/23 22:59 for that hour. The weather dataframe would just have 1/18/23 22:00, 1/18/23 23:00, etc. and the temperature. By only comparing the first part of it "1/18/23 22:" and exculding the minute value you could add the weather value. copying and pasting the weather values into the spreadsheet or adding them 60 times may be easier but wouldn't work with new training data or if the data was missing a certain timestamp.


**COST CALCULATIONS **





In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

training_data['timestamp'] = pd.to_datetime(training_data['timestamp'])


# Get the start and end dates from the 'Timestamp' column of your data
start_date = training_data['timestamp'].min()
end_date = training_data['timestamp'].max()

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 35.76,
	"longitude": -79.02,
	"start_date": start_date.strftime('%Y-%m-%d'),     #THIS IS THE START DATE, CAN REPLACE THIS WITH A DYNAMIC WAY TO GET THE EARLIEST DATE FROM THE TRAINING DATA
	"end_date": end_date.strftime('%Y-%m-%d'),				 #THIS IS THE END DATE, CAN REPLACE THIS WITH A DYNAMIC WAY TO GET THE LATEST DATE FROM THE TRAINING DATA
	"hourly": "temperature_2m",
	"temperature_unit": "fahrenheit",
	"wind_speed_unit": "mph",
	"precipitation_unit": "inch",
	"timeformat": "unixtime",
	"timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]


# Process hourly data. The order of variables needs to be the same as requested.
hourly_data = response.Hourly()
hourly_temperature_2m = hourly_data.Variables(0).ValuesAsNumpy()

# Create a time range with hourly frequency
hourly_time_range = pd.date_range(
    start=pd.to_datetime(hourly_data.Time(), unit="s"),
    end=pd.to_datetime(hourly_data.TimeEnd(), unit="s"),
    freq=pd.Timedelta(seconds=hourly_data.Interval()),
    closed="left"
)

# Ensure that the arrays have the same length
min_length = min(len(hourly_time_range), len(hourly_temperature_2m))
hourly_data = {"Timestamp": hourly_time_range[:min_length], "temperature_2m": hourly_temperature_2m[:min_length]}
hourly_dataframe = pd.DataFrame(data=hourly_data)


# Convert 'timestamp' column in training_data to datetime
training_data['timestamp'] = pd.to_datetime(training_data['timestamp'])

# Merge hourly data with your existing data DataFrame on the 'timestamp' column
merged_data = pd.merge_asof(training_data, hourly_dataframe[['Timestamp', 'temperature_2m']], left_on='timestamp', right_on='Timestamp', direction='backward')

# Drop the extra 'Timestamp' column
merged_data.drop(columns=['Timestamp'], inplace=True)


# Drop the 'Timestamp' column from the final output if needed
#merged_data = merged_data.drop(columns=['Timestamp'])
merged_data['timestamp'] = merged_data['timestamp'].dt.strftime('%m/%d/%Y %H:%M')


#------------------COST CALCULATIONS -----------------------------------------#


electricity_rate = 0.1213  # the average rate of electricity by duke

# List of appliance columns (excluding timestamp, Total, and temperature_2m)
appliance_columns = merged_data.columns.difference(['timestamp', 'Total', 'temperature_2m'])

# energy consumption in KWH for each appliance in each row
for appliance in appliance_columns:
    # Calculate energy consumption (kWh) for each row
    merged_data[f'{appliance}_Energy'] = merged_data[appliance] / 1000

# cost calculation for each appliance in each row
for appliance in appliance_columns:
    # Calculate cost (dollars) for each row
    merged_data[f'{appliance}_Cost'] = merged_data[f'{appliance}_Energy'] * electricity_rate

# Now, merged_data contains additional columns for energy consumption and costs for each appliance in each row
# merged_data also contains hourly temperature information alongside existing data
merged_data

<ipython-input-7-206ceb33d950>:45: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  hourly_time_range = pd.date_range(


,timestamp,Total,Washer,BlowerGH,Lights,BlowerBed,CompGH,CompBed,Dryer,Recs1,...,BlowerBed_Cost,BlowerGH_Cost,CompBed_Cost,CompGH_Cost,Dryer_Cost,Lights_Cost,Recs1_Cost,Recs2_Cost,Washer_Cost,WaterHeater_Cost
0,01/18/2023 22:00,164.1,0.5,4.6,25.7,1.7,21.7,5.8,0.0,7.1,...,0.000206,0.000558,0.000704,0.002632,0.000000,0.003117,0.000861,0.000679,0.000061,0.0
1,01/18/2023 22:01,164.3,0.5,4.6,25.6,1.7,21.7,5.8,0.0,7.1,...,0.000206,0.000558,0.000704,0.002632,0.000000,0.003105,0.000861,0.000679,0.000061,0.0
2,01/18/2023 22:02,164.1,0.6,4.6,25.8,1.8,21.7,5.8,0.0,7.0,...,0.000218,0.000558,0.000704,0.002632,0.000000,0.003130,0.000849,0.000667,0.000073,0.0
3,01/18/2023 22:03,164.0,0.5,4.5,25.6,1.8,21.6,5.8,0.0,7.1,...,0.000218,0.000546,0.000704,0.002620,0.000000,0.003105,0.000861,0.000679,0.000061,0.0
4,01/18/2023 22:04,164.2,0.6,4.6,25.7,1.7,21.7,5.8,0.0,7.0,...,0.000206,0.000558,0.000704,0.002632,0.000000,0.003117,0.000849,0.000667,0.000073,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5771,01/22/2023 22:11,7091.1,6.4,8.9,41.7,225.5,25.2,629.4,970.1,10.1,...,0.027353,0.001080,0.076346,0.003057,0.117673,0.005058,0.001225,0.000885,0.000776,0.0
5772,01/22/2023 22:12,1931.8,6.0,6.3,43.4,226.8,21.1,628.1,-0.1,7.3,...,0.027511,0.000764,0.076189,0.002559,-0.000012,0.005264,0.000885,0.000704,0.000728,0.0
5773,01/22/2023 22:13,1943.2,6.0,6.4,43.5,227.9,21.2,631.0,-0.1,7.3,...,0.027644,0.000776,0.076540,0.002572,-0.000012,0.005277,0.000885,0.000704,0.000728,0.0
5774,01/22/2023 22:14,7393.9,6.4,9.2,41.9,225.9,25.6,628.0,1026.7,10.3,...,0.027402,0.001116,0.076176,0.003105,0.124539,0.005082,0.001249,0.000910,0.000776,0.0


In [ ]:
merged_data.rename(columns={'temperature_2m': 'Temperature'}, inplace=True)
merged_data.to_csv('/content/drive/MyDrive/combined_training_data_with_weather.csv', index=False)

In [ ]:
new_df = merged_data[['timestamp', 'Main', 'Temperature',]].copy() #/content/drive/MyDrive/

new_df.tail(1000).to_csv('/content/drive/MyDrive/blind_test_with_weather.csv', index=False)

KeyError: "['Main'] not in index"